In [1]:
import geemap
import ee

Map = geemap.Map(center=[30, 112])
Map

Map(center=[30, 112], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(To…

In [3]:
roi = Map.draw_last_feature.geometry()

In [4]:
image = ee.ImageCollection("COPERNICUS/CORINE/V20/100m")\
        .filterDate('2017', '2019').first()

In [5]:
Map.addLayer(image, {}, 'img')